# 1. Import packages

In [1]:
import re
import string
from typing import Final, List

from IPython.core.display import display_html, HTML

from textattack import Attack, AttackArgs, Attacker
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions.classification import UntargetedClassification
from textattack.loggers import CSVLogger
from textattack.search_methods import GreedyWordSwapWIR
from textattack.shared.attacked_text import AttackedText
from textattack.transformations import WordSwap

/home/sypark/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Load model and dataset

In [2]:
# Load from my_model.py and my_dataset.py
from my_model import get_model
from my_dataset import dataset

model = get_model("cuda:1")

# 3. Define `CustomWordSwapEmbedding` class

In [3]:
class CustomWordSwapEmbedding(WordSwap):
    def __init__(self) -> None:
        hexword_regex: Final[re.Pattern] = re.compile(r"^[0-9a-fA-F]{4}$")
        self.replacement_words: List[str] = []

        with open("./models/encryptd_vocab.txt") as vocab_file:
            for vocab in vocab_file:
                self.replacement_words.append(
                    vocab.rstrip()
                ) if hexword_regex.fullmatch(vocab.rstrip()) else None
                continue
            pass

        return super().__init__(string.hexdigits)

    def _get_transformations(
        self, current_text: AttackedText, indices_to_modify: List[int]
    ) -> List[AttackedText]:
        words = current_text.words
        transformed_texts: List[AttackedText] = []

        for i in indices_to_modify:
            # if i < 22:
            #     continue
            
            word_to_replace = words[i]
            replacement_words = self._get_replacement_words(word_to_replace)
            transformed_texts_idx: List[AttackedText] = []

            for r in replacement_words:
                if r == word_to_replace:
                    continue

                indices: List[int] = []
                new_words: List[str] = []

                indices.append(i)
                new_words.append(r)

                if i < len(words) - 1:
                    post_r = r[2:] + words[i + 1][2:]
                    indices.append(i + 1)
                    new_words.append(post_r)
                    pass

                transformed_texts_idx.append(
                    current_text.replace_words_at_indices(indices, new_words)
                )
                continue

            transformed_texts.extend(transformed_texts_idx)
            continue

        return transformed_texts

    def _get_replacement_words(self, word: str) -> List[str]:
        replacement_words = [w for w in self.replacement_words if w.startswith(word[:2])]

        if word in replacement_words:
            replacement_words.remove(word)
            pass
        
        return replacement_words

    pass

# 4. Construct `textattack.Attack` object

In [4]:
# Construct the four fundamental components of the attack
goal_function = UntargetedClassification(model)
constraints = [
    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.9),
]
transformation = CustomWordSwapEmbedding()
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

textattack: Unknown if model of class <class 'my_model.Classifier'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


# 5. Construct `textattack.Attacker` object

In [5]:
# Attack until 1000 successful attacks are reached
attack_args = AttackArgs(num_successful_examples=100, random_seed=42)
attacker = Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CustomWordSwapEmbedding
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



  1%|          | 1/100 [00:02<04:04,  2.47s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [00:02<04:37,  2.81s/it]

[[6 (100%)]] --> [[8 (100%)]]

12d4 d400 0000 0001 0101 0108 080a [[0a6e]] [[6e29]] 29be bed0 d000 0015 1506 0632 3217 1703 0303 0300 002a 2a9f 9f0a 0a03 0392 921f 1fc4 c457 57a4 a414 14c4 c47e 7e9e 9eb1 b10e 0e59 59ae aeae aef4 f405 050f 0fed edc0 c019 1963 63cc ccec ec8d 8d1c 1cdc dc23 2324 24b0 b022 220a 0a71 7103 030e 0e94 9407 0733 3399 9925

12d4 d400 0000 0001 0101 0108 080a [[0a1a]] [[1a29]] 29be bed0 d000 0015 1506 0632 3217 1703 0303 0300 002a 2a9f 9f0a 0a03 0392 921f 1fc4 c457 57a4 a414 14c4 c47e 7e9e 9eb1 b10e 0e59 59ae aeae aef4 f405 050f 0fed edc0 c019 1963 63cc ccec ec8d 8d1c 1cdc dc23 2324 24b0 b022 220a 0a71 7103 030e 0e94 9407 0733 3399 9925




...

--------------------------------------------- Result 98 ---------------------------------------------
[[2 (100%)]] --> [[11 (99%)]]

d3aa aa05 [[0562]] [[623b]] 3b9a 9a8f 8f8b 8b4d 4dae ae8f 8f02 0280 800d 0dc2 c27e 7eb7 b756 5689 89c0 c01f 1f5b 5b60 6067 6744 444d 4df2 f2d5 d54a 4ae6 e6f7 f749 4981 811f 1f05 0514 14d3 d3a9 a90e 0ef7 f730 309f 9f3e 3e1a 1a59 59ff ffdf df71 71c2 c244 4452 529c 9c3c 3c05 050a 0af7 f783 83c3 c37a 7a5e 5e38 385b 5b19 19a4 a4e1

d3aa aa05 [[0582]] [[823b]] 3b9a 9a8f 8f8b 8b4d 4dae ae8f 8f02 0280 800d 0dc2 c27e 7eb7 b756 5689 89c0 c01f 1f5b 5b60 6067 6744 444d 4df2 f2d5 d54a 4ae6 e6f7 f749 4981 811f 1f05 0514 14d3 d3a9 a90e 0ef7 f730 309f 9f3e 3e1a 1a59 59ff ffdf df71 71c2 c244 4452 529c 9c3c 3c05 050a 0af7 f783 83c3 c37a 7a5e 5e38 385b 5b19 19a4 a4e1




[Succeeded / Failed / Skipped / Total] 98 / 0 / 1 / 99:  98%|█████████▊| 98/100 [08:27<00:10,  5.18s/it]

--------------------------------------------- Result 99 ---------------------------------------------
[[4 (100%)]] --> [[3 (99%)]]

e661 6124 24a1 a108 0839 397d 7d9a 9a1c 1c3d 3d80 8010 1000 00fe fe69 69f3 f300 0000 0001 0101 [[0108]] [[080a]] [[0a69]] [[6924]] 241f 1fe8 e801 0112 12bc bcef ef8c 8c5e 5ef6 f62a 2a08 0859 59ae ae98 9804 044a 4a3f 3f1a 1aa9 a963 6355 5546 4677 7747 47a3 a3eb ebc3 c3cd cd2a 2a71 7151 51fb fb90 9089 897f 7f49 49cb cbc4 c424 24d1 d174

e661 6124 24a1 a108 0839 397d 7d9a 9a1c 1c3d 3d80 8010 1000 00fe fe69 69f3 f300 0000 0001 0101 [[01ef]] [[ef0a]] [[0a00]] [[0024]] 241f 1fe8 e801 0112 12bc bcef ef8c 8c5e 5ef6 f62a 2a08 0859 59ae ae98 9804 044a 4a3f 3f1a 1aa9 a963 6355 5546 4677 7747 47a3 a3eb ebc3 c3cd cd2a 2a71 7151 51fb fb90 9089 897f 7f49 49cb cbc4 c424 24d1 d174




[Succeeded / Failed / Skipped / Total] 99 / 0 / 1 / 100:  99%|█████████▉| 99/100 [08:29<00:05,  5.15s/it]

--------------------------------------------- Result 100 ---------------------------------------------
[[6 (100%)]] --> [[9 (100%)]]

ea69 6900 0000 0001 0101 0108 080a 0a08 08e6 e671 7103 0300 [[0050]] [[502a]] 2a8d 8d41 41c1 c1e4 e40e 0ebe be2c 2c3b 3b8b 8b39 3952 5256 56b5 b5f6 f6c9 c918 182c 2cfa fad3 d36f 6fef ef09 09a1 a1b4 b422 2264 648c 8ca5 a55f 5f10 1056 5672 72e1 e1d8 d8d5 d5ff ff2b 2b7e 7e5a 5a6d 6d51 5162 623c 3ccc cce8 e8b1 b16a 6a84 8481 8142

ea69 6900 0000 0001 0101 0108 080a 0a08 08e6 e671 7103 0300 [[0002]] [[022a]] 2a8d 8d41 41c1 c1e4 e40e 0ebe be2c 2c3b 3b8b 8b39 3952 5256 56b5 b5f6 f6c9 c918 182c 2cfa fad3 d36f 6fef ef09 09a1 a1b4 b422 2264 648c 8ca5 a55f 5f10 1056 5672 72e1 e1d8 d8d5 d5ff ff2b 2b7e 7e5a 5a6d 6d51 5162 623c 3ccc cce8 e8b1 b16a 6a84 8481 8142



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 99     |
| Number of failed attacks:     | 

# 6. Display attack results

In [6]:
# Display the attack results and the differences
logger = CSVLogger(color_method="html")
for result in attack_results:
    logger.log_attack_result(result)
    continue

display_html(HTML(logger.df[["original_text", "perturbed_text"]].to_html(escape=False)))

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,12d4 d400 0000 0001 0101 0108 080a 0a6e 6e29 29be bed0 d000 0015 1506 0632 3217 1703 0303 0300 002a 2a9f 9f0a 0a03 0392 921f 1fc4 c457 57a4 a414 14c4 c47e 7e9e 9eb1 b10e 0e59 59ae aeae aef4 f405 050f 0fed edc0 c019 1963 63cc ccec ec8d 8d1c 1cdc dc23 2324 24b0 b022 220a 0a71 7103 030e 0e94 9407 0733 3399 9925,12d4 d400 0000 0001 0101 0108 080a 0a1a 1a29 29be bed0 d000 0015 1506 0632 3217 1703 0303 0300 002a 2a9f 9f0a 0a03 0392 921f 1fc4 c457 57a4 a414 14c4 c47e 7e9e 9eb1 b10e 0e59 59ae aeae aef4 f405 050f 0fed edc0 c019 1963 63cc ccec ec8d 8d1c 1cdc dc23 2324 24b0 b022 220a 0a71 7103 030e 0e94 9407 0733 3399 9925
1,ac4a 4a00 0023 2342 429c 9c1d 1ddd dd2d 2d86 86ee eee2 e2c9 c9d1 d18f 8f71 71ed ed25 2591 9116 16d9 d9a8 a82b 2bb2 b2ab abae ae61 614f 4fe7 e7b3 b3e5 e546 4650,ac4a 4a00 0000 0042 429c 9c1d 1ddd dd2d 2d86 86ee eee2 e2c9 c9d1 d18f 8f71 71ed ed25 2591 9116 16d9 d9a8 a82b 2bb2 b2ab abae ae61 614f 4fe7 e7b3 b3e5 e546 4650
2,e3af af5e 5e96 9698 9874 7470 706a 6a49 4950 5080 8010 1008 0800 0020 20d4 d400 0000 0001 0101 0108 080a 0a0a 0af2 f24c 4cc3 c301 0124 24f9 f97e 7e28 28b2 b281 81a5 a565 6587 8776 76ed ed98 9834 34bc bc44 4496 96b2 b2a4 a450 50d4 d414 1457 57b4 b4d6 d655 557e 7e3b 3ba7 a78c 8cfb fb4d 4db8 b8d5 d563 6358 58f2 f2d3 d32d,e38d 8d5e 5e96 962b 2b74 7470 706a 6a5e 5e50 5080 8010 1008 0800 0020 20d4 d40e 0e00 0001 0101 0137 370a 0a0a 0af2 f24c 4cc3 c352 5224 241e 1e7e 7e28 28b2 b281 81a5 a565 6587 8776 76ed ed98 9834 34bc bc44 4496 96b2 b2a4 a412 12d4 d414 1457 57b4 b4d6 d655 557e 7e3b 3ba7 a78c 8ced ed4d 4db8 b8d5 d563 6358 58f2 f2d3 d32d
3,7e9c 9c00 001d 1d2d 2db2 b24d 4d2d 2d53 5345 4541 4152 5243 4348 4820 202a 2a20 2048 4854 5454 5450 502f 2f31 312e 2e31 310d 0d0a,7e9c 9c00 001d 1d2d 2db2 b24d 4d2d 2d53 5345 4541 4152 5243 4348 4820 202a 2a20 2048 4854 5454 5450 502f 2f31 312e 2e31 310d 0d0a
4,5a9a 9a4f 4f91 91a6 a69a 9aeb eb6c 6cfc fcfa fa6a 6aa4 a425 25f9 f921 2144 445a 5a78 7859 5970 70b4 b4c4 c4bc bc2c 2c27 2701 01e3 e34c 4ce7 e7b8 b880 8091 9157 579a 9a2c 2cf9 f98d 8dd4 d442 42bd bdaf afb4 b409 09c7 c756 56c8 c89b 9b76 7622 22f7 f7c3 c335 3561 612c 2c80 8052 524f 4fda dae0,5a00 004f 4ff0 f0a6 a69a 9a50 5002 02fc fcfa fa7c 7ca4 a425 25f9 f921 2144 445a 5a78 7859 5970 70b4 b4c4 c4c8 c82c 2c27 2701 01e3 e34c 4ce7 e7b8 b86c 6c91 9157 579a 9a2c 2c4b 4b8d 8d96 9642 42bd bdaf af1d 1d09 09c7 c756 565b 5b9b 9b76 7622 22f7 f7c3 c335 3561 612c 2c80 8052 524f 4fda dae0
5,01bb bb00 00da daa2 a2c9 c90c 0c3e 3e8c 8cd5 d578 78a2 a2f6 f6a8 a825 25aa aa7f 7f69 696f 6f28 28b6 b61a 1a77 770e 0e93 9391 91fe fe5a 5a92 92ca ca14 1407 07d1 d1d6 d6e0 e02a 2a1e 1e7f 7f5b 5b5f 5f75 7570 7072 72ee ee17 17b3 b3fb fbda da36 36e9 e9ed edba ba01 01da da01 01c3 c311 1184 8475 75bc bc09 0933 339c 9cce ce8f,0165 6500 00da daa2 a2c9 c90c 0c3e 3e8c 8cd5 d578 78a2 a2f6 f6a8 a825 25aa aa7f 7f69 696f 6f28 28b6 b61a 1a77 770e 0e93 9391 91fe fe5a 5a92 92ca ca14 1407 07d1 d1d6 d6e0 e02a 2a1e 1e7f 7f5b 5b5f 5f75 7570 7072 72ee ee17 17b3 b3fb fbda da36 36e9 e9ed edba ba01 01da da01 01c3 c311 1184 8475 75bc bc09 0933 339c 9cce ce8f
6,c4ed ed00 0000 0001 0101 0108 080a 0a00 0018 1802 0208 085e 5e76 76c6 c64c 4c17 1703 0303 0300 0025 2500 0000 0000 0000 0000 0000 0000 0060 6022 22b9 b96f 6f35 3553 53dd dd65 65fd fd56 563c 3ce3 e372 720c 0c77 7714 1414 1441 41ad ad2b 2bdf dfe2 e2e1 e137 37d2 d2e7 e740 4022 2209 0934,c4ed ed00 0000 0001 0101 0108 080a 0a00 007b 7b02 0208 085e 5e76 76c6 c64c 4c17 1703 0303 0300 0025 2500 0000 0000 0000 0000 0000 0000 0060 6022 22b9 b96f 6f35 3553 53dd dd65 65fd fd56 563c 3ce3 e372 720c 0c77 7714 1414 1441 41ad ad2b 2bdf dfe2 e2e1 e137 37d2 d2e7 e740 4022 2209 0934
7,cf53 5300 005b 5b72 72e3 e34e 4efe fe3c 3ced ed86 8692 9258 583a 3a00 0096 96ff ff31 31d2 d216 16a9 a9ff ff62 6235 3596 96d5 d57a 7aeb eb45 45ac ac9b 9b3f 3f5f 5fde defe fed8 d898 985d 5d1a 1ad9 d971 7151 5151 51c4 c49b 9bca ca46 46ac ac88 88fb fbec ecdd dd4a 4a72 7247 477e 7eb1 b14f 4f6e 6ebd bdb4 b4ae ae01 0101 01e7,cf53